# Validation

# 1. Introduction : pourquoi valider ?

Parce qu'on a des choix à faire ! Il faut choisir, déjà, la ou les classes de modèles. Il y en a un grand nombre : $k$ plus proches voisins, arbres de décision, forêt aléatoire, régressions linéaires, régressions logistiques, "support vector machines" (SVM),  réseaux de neurones...

Il faut aussi choisir les _hyperparamètres_ des modèles. Un hyperparamètre est un paramètre qui n'évolue pas au cours de l'entraînement, par exemple : le nombre de voisins considérés par un modèle de $k$ plus proches voisins (par défaut $5$ dans la librairie `scikit-learn`).

Avoir une vague idée du fonctionnement des différents modèles et du rôle joué par les différents hyperparamètres peut nous aider à faire ces choix. Cependant, le mieux reste de mesurer objectivement les performances. C'est précisément le rôle de la validation : mesurer les performances de différentes classes de modèles et de différents hyperparamètres afin de sélectionner la meilleure option.



# 2. Valider avec un jeu de validation

Pour ce premier exercice pratique, nous allons nous concentrer sur un nouveau problème : celui de prédire le taux de victoire d'un pokemon dans un duel. On prédit un pourcentage, c'est-à-dire un nombre, on a donc affaire à un problème de **régression**.

Importez les jeux de données `pokedex.csv` et `combats.csv`. Étudiez leur structure en affichant les dix premières lignes de chaque jeu de données.

In [1]:
import pandas

pokedex = pandas.read_csv("pokedex.csv", sep=";")
pokedex = pokedex.set_index("NUMERO")
pokedex

,NOM,TYPE_1,TYPE_2,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,LEGENDAIRE
NUMERO,,,,,,,,,,,
1,Bulbizarre,Herbe,Poison,45,49,49,65,65,45,1,FAUX
2,Herbizarre,Herbe,Poison,60,62,63,80,80,60,1,FAUX
3,Florizarre,Herbe,Poison,80,82,83,100,100,80,1,FAUX
4,Mega Florizarre,Herbe,Poison,80,100,123,122,120,80,1,FAUX
5,Salamèche,Feu,NaN,39,52,43,60,50,65,1,FAUX
...,...,...,...,...,...,...,...,...,...,...,...
796,Diancie,Roche,Fée,50,100,150,100,150,50,6,VRAI
797,Mega Diancie,Roche,Fée,50,160,110,160,110,110,6,VRAI
798,Hoopa confiné,Psy,Spectre,80,110,60,150,130,70,6,VRAI


In [2]:
combats = pandas.read_csv("combats.csv")
combats

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
...,...,...,...
49995,707,126,707
49996,589,664,589
49997,303,368,368
49998,109,89,109


Manque-t-il des données dans la pokedex ? Si oui, comment traiter ces données manquantes ?

In [3]:
pokedex.isna().any()

NOM                         True
TYPE_1                     False
TYPE_2                      True
POINTS_DE_VIE              False
POINTS_ATTAQUE             False
POINTS_DEFFENCE            False
POINTS_ATTAQUE_SPECIALE    False
POINT_DEFENSE_SPECIALE     False
POINTS_VITESSE             False
NOMBRE_GENERATIONS         False
LEGENDAIRE                 False
dtype: bool

_Il manque au moins un nom et au moins un Type 2. Il est normal que des pokemons n'ait pas de Type 2, mais tous les pokemons devraient avoir un nom._

In [4]:
pokedex[pokedex["NOM"].isna()]

,NOM,TYPE_1,TYPE_2,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,LEGENDAIRE
NUMERO,,,,,,,,,,,
63,NaN,Combat,NaN,65,105,60,60,70,95,1,FAUX


_Parmi les pokemons de type Combat de la première génération, un seul a 65 points de vie : Colossinge._

In [5]:
pokedex.loc[62, "NOM"] = "Colossinge"

# Alternative
pokedex["NOM"] = pokedex["NOM"].fillna("Colossinge")

In [6]:
# Vérifions qu'il n'y a plus de données manquantes dans la colonne NOM
pokedex["NOM"].isna().sum()

0

Ajoutez trois nouvelles colonnes au dataframe `pokedex` à partir des données dans `combats` : le nombre de combats menés, le nombre de combats gagnés, et le pourcentage de combats gagnés. Attention, certains pokémons n'ont jamais combattu...

In [7]:
# Avec GroupBy
nb_combats = combats.groupby("First_pokemon").count()["Winner"] + combats.groupby("Second_pokemon").count()["Winner"]

# Avec value_count
nb_combats = combats.value_counts("First_pokemon") + combats.value_counts("Second_pokemon")

nb_combats

1      133
2      121
3      132
4      125
5      112
      ... 
796    105
797    131
798    119
799    144
800    121
Name: count, Length: 784, dtype: int64

In [8]:
pokedex["NB_COMBATS"] = nb_combats
pokedex

,NOM,TYPE_1,TYPE_2,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,LEGENDAIRE,NB_COMBATS
NUMERO,,,,,,,,,,,,
1,Bulbizarre,Herbe,Poison,45,49,49,65,65,45,1,FAUX,133.0
2,Herbizarre,Herbe,Poison,60,62,63,80,80,60,1,FAUX,121.0
3,Florizarre,Herbe,Poison,80,82,83,100,100,80,1,FAUX,132.0
4,Mega Florizarre,Herbe,Poison,80,100,123,122,120,80,1,FAUX,125.0
5,Salamèche,Feu,NaN,39,52,43,60,50,65,1,FAUX,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...
796,Diancie,Roche,Fée,50,100,150,100,150,50,6,VRAI,105.0
797,Mega Diancie,Roche,Fée,50,160,110,160,110,110,6,VRAI,131.0
798,Hoopa confiné,Psy,Spectre,80,110,60,150,130,70,6,VRAI,119.0


In [9]:
pokedex["NB_COMBATS"].isna().sum()

16

In [10]:
pokedex["NB_COMBATS"].fillna(0, inplace=True)
pokedex["NB_COMBATS"].isna().sum()



0

In [11]:
nb_wins = combats.value_counts("Winner")
pokedex["NB_COMBATS_GAGNES"] = nb_wins
pokedex["NB_COMBATS_GAGNES"].fillna(0, inplace=True)
# Alternative : pokedex["NB_COMBATS_GAGNES"] = pokedex["NB_COMBATS_GAGNES"].fillna(0)
pokedex["NB_COMBATS_GAGNES"]

NUMERO
1       37.0
2       46.0
3       89.0
4       70.0
5       55.0
       ...  
796     39.0
797    116.0
798     60.0
799     89.0
800     75.0
Name: NB_COMBATS_GAGNES, Length: 800, dtype: float64

In [12]:
pokedex["POURCENTAGE_VICTOIRE"] = pokedex["NB_COMBATS_GAGNES"]/pokedex["NB_COMBATS"]
pokedex["POURCENTAGE_VICTOIRE"]

NUMERO
1      0.278195
2      0.380165
3      0.674242
4      0.560000
5      0.491071
         ...   
796    0.371429
797    0.885496
798    0.504202
799    0.618056
800    0.619835
Name: POURCENTAGE_VICTOIRE, Length: 800, dtype: float64

Quel est le Pokémon qui a gagné le plus de combats ?

In [13]:
# Afficher le dataframe trié
pokedex["NB_COMBATS_GAGNES"].sort_values(ascending=False)

# Alternative : récupérer l'indice du max
id_meilleur_pokemon = pokedex["NB_COMBATS_GAGNES"].idxmax()
pokedex.loc[id_meilleur_pokemon]

NOM                          Mewtwo
TYPE_1                          Psy
TYPE_2                          NaN
POINTS_DE_VIE                   106
POINTS_ATTAQUE                  110
POINTS_DEFFENCE                  90
POINTS_ATTAQUE_SPECIALE         154
POINT_DEFENSE_SPECIALE           90
POINTS_VITESSE                  130
NOMBRE_GENERATIONS                1
LEGENDAIRE                     VRAI
NB_COMBATS                    164.0
NB_COMBATS_GAGNES             152.0
POURCENTAGE_VICTOIRE       0.926829
Name: 163, dtype: object

Existe-t-il un pokémon qui n'a jamais gagné en ayant combattu au moins une fois ?

In [14]:
pokedex[(pokedex["NB_COMBATS_GAGNES"] == 0) & (pokedex["NB_COMBATS"] > 0)]

,NOM,TYPE_1,TYPE_2,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,LEGENDAIRE,NB_COMBATS,NB_COMBATS_GAGNES,POURCENTAGE_VICTOIRE
NUMERO,,,,,,,,,,,,,,
231,Caratroc,Insecte,Roche,20,10,230,10,230,5,2,FAUX,135.0,0.0,0.0


Affichez les corrélations entre les différentes variables. Quelles variables semblent utiles pour prédire le pourcentage de victoire ?

In [15]:
pokedex.corr(numeric_only=True)

,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,NB_COMBATS,NB_COMBATS_GAGNES,POURCENTAGE_VICTOIRE
POINTS_DE_VIE,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952,0.058683,-0.040668,0.237030,0.261602
POINTS_ATTAQUE,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240,0.051451,0.014019,0.479151,0.502825
POINTS_DEFFENCE,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227,0.042419,0.067633,0.125996,0.114565
POINTS_ATTAQUE_SPECIALE,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018,0.036437,-0.007411,0.451729,0.481445
POINT_DEFENSE_SPECIALE,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133,0.028486,0.030911,0.295788,0.302422
POINTS_VITESSE,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000,-0.023121,-0.006253,0.887872,0.938055
NOMBRE_GENERATIONS,0.058683,0.051451,0.042419,0.036437,0.028486,-0.023121,1.000000,0.027988,0.028430,0.022987
NB_COMBATS,-0.040668,0.014019,0.067633,-0.007411,0.030911,-0.006253,0.027988,1.000000,0.292286,-0.039430
NB_COMBATS_GAGNES,0.237030,0.479151,0.125996,0.451729,0.295788,0.887872,0.028430,0.292286,1.000000,0.980962
POURCENTAGE_VICTOIRE,0.261602,0.502825,0.114565,0.481445,0.302422,0.938055,0.022987,-0.039430,0.980962,1.000000


## 2. a) Ne pas confondre validation et évaluation !

On est souvent tenté de valider avec le jeu d'évaluation, mais cela enlève tout l'intérêt de préparer un jeu d'évaluation séparé du jeu d'entraînement. On va donc créer un jeu de validation spécialement pour l'occasion.

Séparez les données de la `pokedex` en trois dataframes : `pokedex_train` (60% des observations), `pokedex_validation` (20% des observations) et `pokedex_test` (20 % des observations). Vous pouvez le faire à la main, mais vous pouvez aussi utiliser `train_test_split` de la librairie `scikit-learn` ;)

In [16]:
from sklearn.model_selection import train_test_split

pokedex = pokedex[pokedex["POURCENTAGE_VICTOIRE"].notna()]

train_data, other_data = train_test_split(pokedex, train_size=0.6, random_state=42)
test_data, validation_data = train_test_split(other_data, test_size=0.5, random_state=42)

Séparez maintenant vos trois dataframes en `X` et `Y`. Dans notre cas, les variables prédictives sont :
- les points de vie
- le niveau d'attaque
- le niveau de défense
- le niveau d'attaque spéciale
- le niveau de défense spéciale
- la vitesse
- la génération du Pokémon

Et la variable cible (la prédiction) est :
- le pourcentage de victoire

In [17]:
X_columns = ["POINTS_DE_VIE", "POINTS_ATTAQUE", "POINTS_DEFFENCE", "POINTS_ATTAQUE_SPECIALE","POINT_DEFENSE_SPECIALE", "POINTS_VITESSE", "NOMBRE_GENERATIONS"]
X_train = train_data[X_columns]
X_test = test_data[X_columns]
X_validation = validation_data[X_columns]

Y_train = train_data["POURCENTAGE_VICTOIRE"]
Y_test = test_data["POURCENTAGE_VICTOIRE"]
Y_validation = validation_data["POURCENTAGE_VICTOIRE"]

## 2. b) Différentes classes de modèles

Avec `scikit-learn` entraînez les modèles suivants sur les données d'entraînement et évaluez-les avec les données de validation.

Un arbre de décision (`DecisionTreeRegressor`) :

In [18]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train, Y_train)
dtr.score(X_validation, Y_validation)

0.9246293189912455

In [19]:
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt

# plt.figure(dpi=1000)
# plot_tree(dtr)

Une forêt aléatoire (`RandomForestRegressor`) :

In [20]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train, Y_train)
rfr.score(X_validation, Y_validation)

0.951121121088516

Une régression linéaire (`LinearRegression`) :

In [21]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, Y_train)
lr.score(X_validation, Y_validation)

0.9142051068511403

Une régression "Lasso" (`Lasso`) :

In [22]:
from sklearn.linear_model import Lasso

lr = Lasso()
lr.fit(X_train, Y_train)
lr.score(X_validation, Y_validation)

0.900836050024571

Lequel de ces modèles semble le plus performant ?

_RandomForestRegressor_

# 3 La validation croisée

Créer un jeu de validation, c'est bien, mais ça limite la quantité de données qu'on peut utiliser pour l'entraînement. Une alternative est de faire de la validation croisée.

## 3. a) Choisir un hyperparamètre avec une "Grid Search"

La classe de modèle `Lasso` accepte plusieurs hyperparamètres, parmi eux `alpha`. Sans rentrer dans les détails de ce que signifient ces hyperparamètres, on souhaiterait choisir la meilleure valeur possible. On va donc utiliser une méthode de recherche exhaustive, aussi appelée **grid search**.

Utilisez la méthode `GridSearchCV` de `scikit-learn` pour trouver la meilleure valeur pour `alpha`.

In [23]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {"alpha": np.linspace(0.5, 10, 10)}

lasso = Lasso()
gsc = GridSearchCV(lasso, param_grid, cv=5)
gsc.fit(X_train,Y_train)
gsc.best_params_

{'alpha': 0.5}

In [24]:
lasso2 = Lasso(alpha=0.5)
lasso2.fit(X_train, Y_train)
lasso2.score(X_validation, Y_validation)

0.9131467246073469

Finalement, évaluons le meilleur modèle sur les données de test :

In [25]:
rfr.score(X_test, Y_test)

0.9318473562556767